# Climate QA Agent - Demo Notebook

This notebook demonstrates the Climate QA Agent system, which combines:
- **Multi-turn conversation** with memory
- **Retrieval-Augmented Generation (RAG)** for documentation queries
- **Climate data analysis tools** for numerical computations
- **Router-based agent** for automatic capability selection

## Setup

Before running this notebook:
1. Install dependencies: `pip install -r ../requirements.txt`
2. Set up your `.env` file with `HF_TOKEN`
3. Download data files (see `../data/README.md`)

In [ ]:
# Load environment variables
import os
import sys
from dotenv import load_dotenv

# Add parent directory to path for imports
sys.path.insert(0, os.path.dirname(os.getcwd()))

# Load .env file
load_dotenv(os.path.join(os.path.dirname(os.getcwd()), '.env'))

# Verify HF_TOKEN is set (don't print the actual token!)
assert os.getenv('HF_TOKEN'), "Please set HF_TOKEN in your .env file"
print("Environment loaded successfully.")

## Section 1: Basic LLM Chat

Load the model and demonstrate single-turn chat.

In [ ]:
from src.llm import load_model, generate_response

# Load model (this may take a moment)
model, tokenizer = load_model()

In [ ]:
# Single-turn chat example
response = generate_response("Please introduce yourself in one sentence.")
print("Assistant:", response)

## Section 2: Conversation Memory

Demonstrate multi-turn conversation with history.

In [ ]:
from src.memory import hf_chat, create_chat_session

# Create a new chat session
chat_history = create_chat_session()

# First turn
response1 = hf_chat("What is artificial intelligence?", chat_history)
print("User: What is artificial intelligence?")
print("Assistant:", response1)
print()

In [ ]:
# Follow-up (uses conversation context)
response2 = hf_chat("Summarize your previous answer in one sentence.", chat_history)
print("User: Summarize your previous answer in one sentence.")
print("Assistant:", response2)

## Section 3: Retrieval-Augmented Generation (RAG)

Demonstrate document retrieval and citation-based answering.

In [ ]:
from src.rag import build_vectorstore, rag_answer

# Build the vector store (loads corpus and creates embeddings)
vectorstore = build_vectorstore()

In [ ]:
# RAG query example
result = rag_answer("What does t2m mean and what unit is it in?")
print("\nAnswer:")
print(result["answer"])

In [ ]:
# Another RAG query
result = rag_answer("What is the spatial and temporal coverage of the dataset?")
print("\nAnswer:")
print(result["answer"])

## Section 4: Climate Data Analysis Tools

Demonstrate the computational tools for NetCDF data analysis.

In [ ]:
from src.tools import load_datasets, inspect_dataset, compute_stat

# Load datasets
datasets = load_datasets()

In [ ]:
# Inspect a dataset
info = inspect_dataset("t2m")
print("Dataset Info:")
print(info)

In [ ]:
# Compute statistics
print("Mean temperature (Celsius):")
print(compute_stat("t2m", metric="mean", units="C"))

print("\nTotal precipitation:")
print(compute_stat("tp", metric="sum"))

## Section 5: Complete Agent System

Demonstrate the full router-based agent that automatically selects capabilities.

In [ ]:
from src.agent import run_agent
import json

# Test questions
test_questions = [
    "What does t2m mean and what unit is it measured in?",
    "What is the average temperature (t2m) in Celsius?",
    "What does t2m mean and what is its average value in January?",
    "What is the temporal coverage of the dataset and what is the total precipitation?",
]

In [ ]:
# Run agent on test questions
for q in test_questions:
    print("=" * 80)
    print(f"Question: {q}")
    
    result = run_agent(q)
    
    print("\nRouter Plan:")
    print(json.dumps(result["plan"], indent=2))
    
    print("\nFinal Answer:")
    print(result["final_answer"])
    print()

## Interactive Mode

Try your own questions!

In [ ]:
# Your custom question
your_question = "What is the maximum temperature in the dataset?"

result = run_agent(your_question)
print(f"Question: {your_question}")
print(f"\nPlan: {json.dumps(result['plan'], indent=2)}")
print(f"\nAnswer: {result['final_answer']}")